In [1]:
from process_bigraph import Composite 

from biosimulators_processes import CORE 

Cannot register SimulariumSmoldynStep. Error:
**
No module named 'simulariumio'
**
Cannot register MongoDatabaseEmitter. Error:
**
No module named 'simulariumio'
**


In [2]:
model_fp = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator/BIOMD0000000012_url.xml'

doc = {
    'dFBA': {
        '_type': 'process',
        'address': 'local:dfba-process',
        'config': {
            'model': {
                'model_source': model_fp
            },
            'simulator': 'copasi',
            'start': 0,
            'stop': 10,
            'steps': 100
        },
        'inputs': {},
        'outputs': {
            'solution': ['solution_store']
        }
    },
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'solution': 'tree[float]'
            }
        },
        'inputs': {
            'solution': ['solution_store']
        }
    }
}


spec = doc.copy()

In [ ]:
comp = Composite(
    config={'state': spec},
    core=CORE
)

In [ ]:
comp.run(3)

In [1]:
import os 
from pathlib import Path

from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import cobra
from cobra.io import load_model as load_cobra, read_sbml_model
from basico import * 
import numpy as np
from tqdm import tqdm

from biosimulators_processes.helpers import generate_reaction_mappings


# set up models
model_file = '/Users/alexanderpatrie/Desktop/repos/biosimulator-processes/test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator/BIOMD0000000012_url.xml'

# set up cobra model
data_dir = Path(os.path.dirname(model_file))
path = data_dir / model_file.split('/')[-1]
cobrapy = read_sbml_model(str(path.resolve()))  # load_cobra('textbook')

# set up copasi model
copasi = load_model(model_file)

# global store TODO: let process bigraph handle this
concentrations = {}


def add_dynamic_bounds(fba_model, utc_model, y, mappings):
    """
    # 1. get reaction mappings 
    # 2. y = get_species(model=copasi).initial_concentration.values
    # 3. for each species in y, calculate max import (TODO: make this specific)
    # 4. for each species mapping, use the dict val (reaction name) to say fba_model.reactions.get_by_id(reaction_name) = max_import
    """
    # Zip species names and their corresponding concentrations for iteration
    species_concentrations = y  # dict(zip(species_names, y))
    for mapping in mappings:
        for species, reaction_name in mapping.items():  # Each mapping is a dictionary like {'LacI mRNA': 'degradation of LacI transcripts'}
            if species in species_concentrations:
                concentration = species_concentrations[species]
                rxn_id = reaction_name.replace(" ", "_")
                for reaction in fba_model.reactions:
                    # TODO: make this more specific
                    if "degradation" in reaction_name:
                        max_import = -10 * concentration / (5 + concentration)
                        fba_model.reactions.get_by_id(reaction.id).lower_bound = max_import
                    elif "transcription" in reaction_name:
                        max_import = 5 * concentration / (3 + concentration)
                        fba_model.reactions.get_by_id(reaction.id).lower_bound = max_import
                    elif "translation" in reaction_name:
                        max_import = 8 * concentration / (4 + concentration)
                        fba_model.reactions.get_by_id(reaction.id).lower_bound = max_import


def apply_mm_kinetics(concentration, Vmax, Km):
    """Apply Michaelis-Menten kinetics to calculate the flux."""
    return (Vmax * concentration) / (Km + concentration)


def dynamic_system(t, y, fba_model, utc_model, mappings):
    """
    Calculate the time derivative of external species using specific reaction mappings.

    Parameters:
        t: Current time
        y: Array of species concentrations at time t
        fba_model: The FBA model (e.g., COBRA model)
        utc_model: The UTC model (e.g., COPASI model)
        mappings: List of mappings from species names to reactions
        species_names: List of species names corresponding to y
        num_steps: granularity of output

    Returns:
        Fluxes calculated based on current concentrations
    
    # 1. run copasi utc (update model!) 
    # 2. run get_species(model=copasi).concentration.to_dict().values()
    # 3. assign these to y = #2
    # 4. iteratively run: for species in y: DO THE REST
    """
    # run copasi for a single interval
    run_time_course(model=copasi, update_model=True, intervals=1)

    # set y to this value array
    y = get_species(model=copasi)['concentration'].to_dict()
    
    # update global store TODO: do this with process bigraph
    global concentrations 
    for name, value in y.items():
        concentrations[name].append(value)

    # update the FBA model's reaction bounds using species concentrations and mappings
    add_dynamic_bounds(fba_model, utc_model, y, mappings)

    # Run FBA with updated bounds and calculate fluxes
    cobra.util.add_lp_feasibility(fba_model)
    feasibility = cobra.util.fix_objective_as_constraint(fba_model)
    
    # Example of reactions to optimize (can vary based on your specific model)
    reaction_list = [rxn.id for rxn in fba_model.reactions]
    obj_directions = ['max' for _ in reaction_list]  # TODO: make this more fine-grained
    lex_constraints = cobra.util.add_lexicographic_constraints(fba_model, reaction_list, obj_directions)

    # scale fluxes by the concentration for each species at time t: either for biomass or by mm kinetics
    fluxes = []
    for species, concentration in y.items():
        if species in concentrations.keys():
            # handle scaling by biomass
            if species == 'biomass':
                biomass_concentration = y['biomass']
                flux = lex_constraints.values * biomass_concentration
                fluxes.append(flux)
            else:
                # handle scaling by mm kinetics TODO: change this to be dynamic input
                Vmax, Km = 10, 5
                flux = apply_mm_kinetics(concentration, Vmax, Km)
                fluxes.append(flux)
        else:
            # give raw values if not TODO: handle this differently
            fluxes.append(lex_constraints.values)

    # update progress bar
    if dynamic_system.pbar is not None:
        dynamic_system.pbar.update(1)
        dynamic_system.pbar.set_description(f't = {t:.3f}')

    return fluxes


dynamic_system.pbar = None


def infeasible_event(t, y):
    """
    Determine solution feasibility.

    Avoiding infeasible solutions is handled by solve_ivp's built-in event detection.
    This function re-solves the LP to determine whether or not the solution is feasible
    (and if not, how far it is from feasibility). When the sign of this function changes
    from -epsilon to positive, we know the solution is no longer feasible.

    """
    with cobrapy:
        add_dynamic_bounds(cobrapy, copasi, y)
        cobra.util.add_lp_feasibility(cobrapy)
        feasibility = cobra.util.fix_objective_as_constraint(cobrapy)

    return feasibility - infeasible_event.epsilon



# 1. run get_species(model=copasi).initial_concentration.to_dict().
# 2. assign 1 to y0
# 3. gen reaction mapping
# 4. specify num steps
# 5. add 3 and 4 to args below


# set infeasibility params TODO: make this arg
infeasible_event.epsilon = 1E-6
infeasible_event.direction = 1
infeasible_event.terminal = True

# time params
start = 0
stop = 10
steps = 100
ts = np.linspace(start, stop, steps)
initial_concentrations = get_species(model=copasi)['initial_concentration'].to_dict()
concentrations = {name: [initial_concentrations[name]] for name in initial_concentrations.keys()}
y0 = list(initial_concentrations.values())
mappings = generate_reaction_mappings(list(initial_concentrations.keys()), cobrapy)

# run and get solution
with tqdm() as pbar:
    dynamic_system.pbar = pbar

    sol = solve_ivp(
        fun=dynamic_system,
        events=[infeasible_event],
        t_span=(ts.min(), ts.max()),
        y0=y0,
        t_eval=ts,
        rtol=1e-6,
        atol=1e-8,
        method='BDF',
        args=(cobrapy, copasi, mappings)
    )

Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction1 "degradation of LacI transcripts">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction Reaction1 "degradation of LacI transcripts">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction2 "degradation of TetR transcripts">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction Reaction2 "degradation of TetR transcripts">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction3 "degradation of CI transcripts">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction Reaction3 "degradation of CI transcripts">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction4 "translation of LacI">'
Missing upper flux bound se

Cannot register SimulariumSmoldynStep. Error:
**
No module named 'simulariumio'
**
Cannot register MongoDatabaseEmitter. Error:
**
No module named 'simulariumio'
**


0it [00:00, ?it/s]


NameError: name 'species_names' is not defined